# Retrieve data from eNanomapper database
https://search.data.enanomapper.net/

- This notebook uses Apache Solr API and AMBIT REST API  (see Swagger interactive documentation at http://enanomapper.github.io/API/ )

In [1]:
import pandas as pd
import os.path
import numpy as np
import datetime, time
import urllib3
import json
import sys
import ipywidgets as widgets

import logging
from logging.config import fileConfig
fileConfig('logging_endpoints_config.ini')

global logger
logger = logging.getLogger()

%run units.py

logger.debug('Started at %s \t%s',os.name, datetime.datetime.now())

urllib3.disable_warnings()

In [2]:
%run solrapi.py

In [3]:
%run enmapi.py

In [4]:
http_pool=urllib3.PoolManager()

### Retrieve endpoints using AMBIT REST API

In [5]:
ambit=AMBITQuery(key="study")
print(ambit.root)
data = ambit.get(http_pool)


2019-03-23 13:11:29,130  INFO     Sending query to https://apps.ideaconsult.net/nanoreg1/query/study


https://apps.ideaconsult.net/nanoreg1


2019-03-23 13:11:29,306  INFO     Received response 


In [6]:
#Select endpoint
_sections={}

top_sections = []
for facet in data['facet']:
    #print("{}\t{}\t{}\t{}".format(facet['subcategory'],facet['endpoint'],facet['count'],facet['value']))
    top = facet['subcategory']
    if not top in top_sections:
        top_sections.append(top)
        
    if not top in _sections:
        _sections[top] = []
    
    _sections[top].append(facet['endpoint'])

top_widget = widgets.Dropdown(
    options=top_sections,
    value=top_sections[0],
    description='Select:',
    disabled=False,
)
display(top_widget)


Dropdown(description='Select:', options=('ECOTOX', 'P-CHEM', 'TOX'), value='ECOTOX')

In [7]:
top=top_widget.value
category_widget = widgets.Dropdown(
    options=_sections[top],
    value=_sections[top][1],
    description='Endpoint:',
    disabled=False,
)
display(category_widget)

Dropdown(description='Endpoint:', index=1, options=('EC_ALGAETOX_SECTION', 'EC_DAPHNIATOX_SECTION', 'EC_SOILDW…

#### Setup the query

In [8]:
freetext_widget=widgets.Text(
    value='NM220,NM101',
    description='Search',
    disabled=False
)
freetext_widget

Text(value='NM220,NM101', description='Search')

In [9]:
materialfilter=freetext_widget.value
if "" == materialfilter:
    materialfilter=None
    
section=category_widget.value
fields=None
if section in _fields:
    fields=_fields[section]

endpoint="*"
if section in _endpoint:
    endpoint=_endpoint[section]
    
settings = {'endpointfilter': ' effectendpoint_s: {}'.format(endpoint),
            'studyfilter': ' topcategory_s:({}) AND endpointcategory_s:({})'.format(top,section),            
            'query_guidance': None, 
            'query_organism': None,
            'fields' : fields
           }
    
logger.info(settings)  
logger.info("Free text query: {}".format(materialfilter))

2019-03-23 13:11:42,164  INFO     {'endpointfilter': ' effectendpoint_s: *', 'studyfilter': ' topcategory_s:(ECOTOX) AND endpointcategory_s:(EC_DAPHNIATOX_SECTION)', 'query_guidance': None, 'query_organism': None, 'fields': None}
2019-03-23 13:11:42,165  INFO     Free text query: NM220,NM101


### Run the query

In [10]:

try:
    with open('./config.solr.json') as f:
        solr_services = json.load(f)
except:        
    solr_services = None
    logger.error("Something went wrong")

service_widget = widgets.Dropdown(
    options=solr_services.keys(),
    description='Service:',
    disabled=False,
)
display(service_widget)

name_widget=widgets.Text(
    placeholder='',
    description='User name:',
    disabled=False
)
password_widget = widgets.Password(description='Password:', placeholder='')
display(name_widget)
password_widget

Dropdown(description='Service:', options=('nanoreg1', 'enm'), value='nanoreg1')

Text(value='', description='User name:', placeholder='')

Password(description='Password:', placeholder='')

In [11]:
query=getSolrQuery(settings,url=solr_services[service_widget.value])
rows = []  

#logger.info(settings)
logger.info(query)
rows = sendSolrRequest(settings=settings,http=http_pool,textfilter=materialfilter,query=query,rows=rows,basic_auth="{}:{}".format(name_widget.value,password_widget.value))

2019-03-23 13:11:52,659  INFO     {'url': 'https://solr.ideaconsult.net/solr/nanoreg_shard1_replica1/select', 'fl': 'name_hs,publicname_hs,substanceType_hs,s_uuid_hs,[child parentFilter=filter(type_s:substance) childFilter="filter(type_s:study AND  topcategory_s:(ECOTOX) AND endpointcategory_s:(EC_DAPHNIATOX_SECTION) AND  effectendpoint_s: *)  OR filter(type_s:params AND  topcategory_s:(ECOTOX) AND endpointcategory_s:(EC_DAPHNIATOX_SECTION))  OR filter(type_s:conditions AND  topcategory_s:(ECOTOX) AND endpointcategory_s:(EC_DAPHNIATOX_SECTION)) OR filter(type_s:composition AND component_s:CONSTITUENT)" limit=10000]', 'fq': '', 'q': '{!parent which=type_s:substance}'}
2019-03-23 13:11:52,661  INFO     Sending query to https://solr.ideaconsult.net/solr/nanoreg_shard1_replica1/select
2019-03-23 13:11:52,845  INFO     {
  "name_hs": "NM-100 (TiO2 50-150 nm)",
  "publicname_hs": "JRCNM01000a",
  "substanceType_hs": "NPO_1486",
  "s_uuid_hs": "NNRG-18280a4a-45e9-adc0-df3b-125397b1255f"
}


In [12]:
#print("Substances: {}".format(len(rows)))
df = pd.DataFrame(rows)
df.to_csv(section+".nosmiles.txt",sep='\t',index=False)
#df.head()
df.head()

In [13]:
import numpy as np
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: red' if v else '' for v in is_max]

for criteria in ["value.lo"]:
    tmp = pd.pivot_table(df, values=criteria, index=['public.name'], columns=['x.oht.top','x.oht.section','x.guidance','value.endpoint','value.endpoint_type','value.qualifier.lo','value.unit'], aggfunc=np.mean).reset_index()
    #tmp.style.highlight_null(null_color='red')
    
    display(tmp.style.apply(highlight_max,subset=top_sections))

In [14]:
tmp=df.groupby(by=["public.name","x.guidance","value.endpoint","value.endpoint_type","value.qualifier.lo","value.unit"]).agg({"value.lo" : ["min","max","mean","std","count"]}).reset_index()
tmp.columns = ["_".join(x) for x in tmp.columns.ravel()]
print("Substances {}".format(tmp.shape[0]))
display(tmp)


Substances 20
